In [0]:
import dataiku
import pandas as pd

# Connect to the DSS instance
client = dataiku.api_client()


In [0]:
# Prepare a list to store results
project_hive_counts = []

# Iterate over all projects
projects = client.list_project_keys()
for project_key in projects:
    project = client.get_project(project_key)
    
    # Get the list of recipes in the project
    recipes = project.list_recipes()
    
    # Count the number of Hive recipes
    hive_count = sum(1 for recipe in recipes if recipe['type'] == 'Hive')

    # Append the result to the list
    project_hive_counts.append({'project_key': project_key, 'hive_recipe_count': hive_count})

# Create a Pandas DataFrame with the results
df = pd.DataFrame(project_hive_counts)
df.sort_values(by="hive_recipe_count", ascending=False)

In [0]:
import dataiku
import pandas as pd
from datetime import datetime

# Connect to the DSS instance
client = dataiku.api_client()

# Prepare a list to store results
project_info = []

# Iterate over all projects
projects = client.list_project_keys()
for project_key in projects:
    try:
        project = client.get_project(project_key)
        
        # Get project metadata to extract owner information
        project_metadata = project.get_metadata()
        project_summary = project.get_summary()
        last_modified_timestamp = project_summary.get('versionTag', None).get('lastModifiedOn', None)
        if last_modified_timestamp:
            last_modified_date = datetime.fromtimestamp(last_modified_timestamp / 1000).strftime('%Y-%m-%d-%H-%M-%S')
        else:
            last_modified_date = 'unknown_date'
        
        # Get the owner username
        #owner_username = project_metadata.get('owner', '')
        
        # Get the list of recipes in the project
        recipes = project.list_recipes()
        
        # Count the number of Hive recipes
        hive_recipes = [recipe for recipe in recipes if recipe['type'].lower() == 'hive']
        hive_count = len(hive_recipes)
        
        
        datasets = project.list_datasets()
        extracted_data = [
        {
            'type': row.get('type'),
            'name': row.get('name'),
            'connection': row.get('params', {}).get('connection'),
            'table': row.get('params', {}).get('table'),
            'catalog': row.get('params', {}).get('catalog'),
            'schema': row.get('params', {}).get('schema'),
            # get the schema too
        }
        for row in datasets
        ]
        
        for d in  project.list_datasets():
            if d.get('type') == 'hive'
        num_hive_datasets = 
        
        
        # Append the result to the list
        project_info.append({
            'PROJECT_KEY': project_key, 
            'NUMBER_OF_HIVE_RECIPES_USED_IN_THIS_PROJECT': hive_count,
            #'PROJECT_OWNER_USERNAME': owner_username,
            #'PROJECT_OWNER_EMAIL': owner_email,
            'DATE_PROJECT_WAS_LAST_MODIFIED': last_modified_date,
            # 'DATE_LAST_JOB_WAS_RUN': last_job_date
        })
    except Exception as e:
        print(f"Error processing project {project_key}: {str(e)}")
        # Add error entry to maintain record of all projects
        project_info.append({
            'PROJECT_KEY': project_key,
            'NUMBER_OF_HIVE_RECIPES_USED_IN_THIS_PROJECT': -1,  # Error indicator
            #'PROJECT_OWNER_USERNAME': "",
            #'PROJECT_OWNER_EMAIL': "",
            'DATE_PROJECT_WAS_LAST_MODIFIED': "unknown",
            # 'DATE_LAST_JOB_WAS_RUN': "",
        })

# Create a Pandas DataFrame with the results
df = pd.DataFrame(project_info)
df
# Display the DataFrame


# Optional: If you want to export to CSV
# df.to_csv('project_hive_recipe_analysis.csv', index=False)

In [0]:
df